In [12]:
#| default_exp classes.DomoRole

In [13]:
#| exporti
from dataclasses import dataclass, field
import httpx

from fastcore.basics import patch_to

import domolibrary.client.DomoAuth as dmda

import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoError as de
import domolibrary.routes.role as role_routes


In [14]:
# | export
@dataclass
class DomoRole:
    auth: dmda.DomoAuth = field(repr = False)

    id: str
    name: str = None
    description: str = None
    is_system_role: int = None

    grant_ls: list[str] = field(default_factory=list)
    membership_ls: list = field(default_factory=list)

    def __post_init__(self):
        self.is_system_role = 1 if self.id <= 5 else 0

    # @classmethod
    # def _from_str(cls, id, name, description=None, auth: DomoAuth = None):

    #     return cls(id=id,
    #             name=name,
    #             description=description,
    #             auth=auth
    #             )

    @classmethod
    def _from_json(cls, obj, auth=dmda.DomoAuth):

        dd = obj
        if not isinstance(dd, util_dd.DictDot):
            dd = util_dd.DictDot(obj)

        return cls(id=dd.id,
                   name=dd.name,
                   description=dd.description,
                   auth=auth
                   )


In [15]:
# | export
@patch_to(DomoRole)
async def get_role_grants(self: DomoRole,
                          auth: dmda.DomoAuth = None,
                          role_id: str = None,
                          debug_api: bool = False,
                          session: httpx.AsyncClient = None,
                          return_raw: bool = False
                          ):

    auth = auth or self.auth
    role_id = role_id or self.id
    res = await role_routes.get_role_grants(auth=auth,
                                            role_id=role_id,
                                            debug_api=debug_api,
                                            session=session)

    if return_raw:
        return res
        
    self.grant_ls = res.response

    return self.grant_ls


In [16]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

domo_role = DomoRole(auth = token_auth, id = 1)
await domo_role.get_role_grants()

['versions.deployment.manage',
 'buzz.admin',
 'dataflow.admin',
 'goal.admin',
 'dataset.admin',
 'workflow.admin',
 'profile.achieve.assign',
 'datascience.notebooks.admin',
 'content.dataapp.edit',
 'dataset.export',
 'user.invite',
 'connector.admin',
 'content.card.analyzerview',
 'fileshare.create',
 'customer.edit',
 'content.variable.edit',
 'pipeline.executor.job.admin',
 'cloud.admin',
 'versions.repository.admin',
 'codeengine.package.manage',
 'appstore.admin',
 'datastore.admin',
 'certifiedcontent.admin',
 'group.admin',
 'tasks.admin',
 'group.edit',
 'app.ddx',
 'queue.create',
 'domoapps.edit',
 'content.export',
 'codeengine.package.admin',
 'domo.system.messages',
 'publish.space.admin',
 'authorization.roles',
 'dataset.manage',
 'goal.distribute',
 'appstore.read',
 'content.card.embed',
 'content.admin',
 'user.session.edit',
 'cloud.override',
 'datastore.create',
 'user.edit',
 'content.card.edit',
 'saasaas.edit',
 'developer.token.manage',
 'publish.subscriber

In [17]:
# | export
@dataclass
class DomoRoles:
    auth: dmda.DomoAuth


In [18]:
# | export

@patch_to(DomoRoles, cls_method=True)
async def get_roles(cls: DomoRoles,
                    auth: dmda.DomoAuth,
                    debug_api: bool = False,
                    session: httpx.AsyncClient = None,
                    return_raw: bool = False
                    ):

    res = await role_routes.get_roles(auth=auth, session=session, debug_api=debug_api)

    if return_raw:
        return res

    role_ls = res.response

    return [ DomoRole._from_json(role, auth) for role in res.response]


In [19]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

await DomoRoles.get_roles(auth=token_auth)

[DomoRole(id=1, name='Admin', description='Full access to everything', is_system_role=1, grant_ls=[], membership_ls=[]),
 DomoRole(id=2, name='Privileged', description='Full access except for editing users and settings owners', is_system_role=1, grant_ls=[], membership_ls=[]),
 DomoRole(id=3, name='Editor', description='Can edit Cards, Pages, DataSets, and Dataflows', is_system_role=1, grant_ls=[], membership_ls=[]),
 DomoRole(id=4, name='Participant', description='Read only access on Cards, Pages DataSets and Dataflows', is_system_role=1, grant_ls=[], membership_ls=[]),
 DomoRole(id=5, name='Social', description='Access limited to social features', is_system_role=1, grant_ls=[], membership_ls=[]),
 DomoRole(id=275763436, name='Super_Admin', description='Full access to everything', is_system_role=0, grant_ls=[], membership_ls=[]),
 DomoRole(id=810756122, name='super_admin', description='created via DomoLibrary', is_system_role=0, grant_ls=[], membership_ls=[]),
 DomoRole(id=1927158482,

In [20]:
#| export
class SearchRole_NotFound(de.DomoError):
    def __init__(self, domo_instance,
                 role_id,
                 message='not found',
                 function_name='search_role'):

        super().__init__(domo_instance=domo_instance, message=message,
                         entity_id=role_id, function_name=function_name)

@patch_to(DomoRoles, cls_method=True)
async def search_role(cls: DomoRoles,
                    role_name: str,
                    auth: dmda.DomoAuth,
                    debug_api: bool = False,
                    session: httpx.AsyncClient = None,
                    return_raw: bool = False
                    ):

    all_roles = await DomoRoles.get_roles( auth=auth)
    
    domo_role = next(
        (role for role in all_roles if role.name == role_name), None)

    if not domo_role:
        raise SearchRole_NotFound(domo_instance=auth.domo_instance, role_id=role_name)

    return domo_role

In [21]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

await DomoRoles.search_role(auth=token_auth, role_name = 'Admin')


DomoRole(id=1, name='Admin', description='Full access to everything', is_system_role=1, grant_ls=[], membership_ls=[])

In [22]:
class CreateRole_Error(de.DomoError):
    def __init__(self, domo_instance,
                 role_id,
                 message,
                 status,
                 function_name='create_role'):

        super().__init__(domo_instance=domo_instance,
                         message=message, status=status,
                         entity_id=role_id,
                         function_name=function_name)


In [23]:
@patch_to(DomoRoles, cls_method=True)
async def create_role(cls: DomoRoles,
                      auth: dmda.DomoAuth,
                      name: str,
                      description: str = None,
                      debug_api: bool = False):

    res = await role_routes.create_role(auth=auth,
                                        name=name,
                                        description=description,
                                        debug_api=debug_api)
    
    if not res.is_success:
        raise CreateRole_Error(
            domo_instance = auth.domo_instance,
            role_id = name,
            message = res.response,
            status = res.status,
        )
    
    return await DomoRoles.search_role(auth = auth, role_name= name)
    

#### sample implementation of create_role

In [24]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

try:
    await DomoRoles.create_role(auth = token_auth, name = 'super_admin', description = 'created via DomoLibrary',)
except Exception as e:
    print(e)

create_role: super_admin :Status 400 - Bad Request at domo-dojo


In [25]:
# | export

@patch_to(DomoRoles, cls_method=True)
async def upsert_role(cls: DomoRoles,
                      auth: dmda.DomoAuth,
                      name: str,
                      description: str = None,
                      debug_api: bool = False,
                      debug_prn: bool = False,
                      session : httpx.AsyncClient=None
                      ):

    domo_role = await DomoRoles.search_role(role_name=name,
                                            auth = auth ,
                                            debug_api=debug_api,
                                            session=session,
                                            )

    if domo_role:
        return domo_role

    if debug_prn:
        print(
            f'No role match -- creating new role in {full_auth.domo_instance}')

    return await DomoRoles.create_role(auth=auth,
                                       name=name, description=description,
                                       debug_api=debug_api, session=session)


In [26]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

await DomoRoles.upsert_role(auth=token_auth, name='super_admin', description='created via DomoLibrary',)


DomoRole(id=810756122, name='super_admin', description='created via DomoLibrary', is_system_role=0, grant_ls=[], membership_ls=[])

In [27]:
# #| export



# async def get_membership(self, auth: DomoAuth = None,
#                             role_id=None, debug_api: bool = False, session: httpx.AsyncClient = None):

#     res = await role_routes.get_role_membership(auth=auth or self.auth,
#                                                 role_id=role_id or self.id,
#                                                 debug_api=debug_api, session=session)
#     if res.status == 200:
#         self.membership_list = res.response.get('users')

#         return {'status': res.status,
#                 'users': res.response.get('users')}

# @classmethod
# async def create_role(cls,
#                         auth: DomoAuth,
#                         name: str,
#                         description: str = None,
#                         debug_api: bool = False):

#     res = await role_routes.create_role(auth=auth,
#                                         name=name,
#                                         description=description,
#                                         debug_api=debug_api)
#     if debug_api:
#         print(res)

#     if res.status == 200:
#         return cls._from_str(id=res.response.get('id'),
#                                 name=res.response.get('name'),
#                                 description=res.response.get('description'),
#                                 auth=auth)
#     return None

# async def update_role_grants(self,
#                                 grant_list: list[str],
#                                 role_id: str = None,
#                                 auth: DomoAuth = None,
#                                 debug_api: bool = False):

#     res = await role_routes.update_role_grants(auth=auth or self.auth,
#                                                 role_id=role_id or self.id,
#                                                 role_grant_list=grant_list,
#                                                 debug_api=debug_api)

#     if res.status == 200 or res.status == 204:
#         res_roles = await role_routes.get_role_grants(auth or self.auth,
#                                                         role_id=role_id or self.id,
#                                                         debug_api=debug_api)
#         return {'status': res.status,
#                 'role_id': role_id or self.id,
#                 'grants': res_roles.response
#                 }

# async def add_user(self, user_list: list[str],
#                     role_id: str = None,
#                     auth: DomoAuth = None, debug_api: bool = False):

#     res = await role_routes.role_membership_add_user(auth=auth or self.auth,
#                                                         role_id=role_id or self.id,
#                                                         user_list=user_list,
#                                                         debug_api=debug_api)
#     if res.status == 204:
#         res_membership = await role_routes.get_role_membership(auth=auth or self.auth,
#                                                                 role_id=role_id or self.id,
#                                                                 debug_api=debug_api)

#         return {'status': res.status,
#                 'users': res_membership.response.get('users')}

#     return res

# async def set_as_default_role(self, debug_api: bool = False):
#     return await role_routes.set_default_role(auth=self.auth,
#                                                 role_id=self.id,
#                                                 debug_api=debug_api)


Bad pipe message: %s [b'\xb2\xbeb\x83\x94\xd6}\x01w\xa9\xda\xb7\x93\x92\xf6L$q \xcd', b'O8\xd4z{']
Bad pipe message: %s [b'\xbcl\x0eg\xd9\xfb\xbda', b'b\x0e\xbf\xd0\xb2\x94\xfa\xad K\x8c:H(\x9c=Q`\x07\x81@\xaen \xc1\x9fO\xd1Z&\xfb\xdb\xdbz']
Bad pipe message: %s [b"\xe6\xfdhx\xad\nL\xaa.\x1a\xde\xc9a\r\xcbP\xab\xc5\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0s\xc0w\x00\xc4\x00\xc3\xc0#\xc0'\x00g\x00@\xc0r\xc0v\x00\xbe\x00\xbd\xc0\n\xc0\x14\x009\x008\x00\x88\x00\x87\xc0\t\xc0\x13\x003\x002\x00\x9a\x00\x99\x00E\x00D\xc0\x07\xc0\x11\xc0\x08\xc0\x12\x00\x16\x00\x13\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P"]
Bad pipe message: %s [b'KS%&\xbfZn\x12\x03F\xca}PQ1\xdb\xb60\x00\x00\xa2\xc0\x14\xc0\n\x009\x008\x007\x006\x00\x88\x00\x87\x00\x86\x00\x85\xc0\x19\x00:\x00\x89\xc0\x0f\xc0\x05\x005\x00\x84\xc0\x13\xc0